In [2]:
import tensorflow as tf
print(tf.__version__)

2.17.0


## 1. Load and Preprocess the Dataset

In [4]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt

In [19]:
# Set the path to your dataset
DATASET_PATH = r'C:\Users\Dell\GAN_Chest_Xray_Synthesis\data\chest_xray'

In [21]:
# Image size and batch size
IMG_SIZE = 128  # Resize images to 128x128 pixels
BATCH_SIZE = 32

In [23]:
# Create data generators for training, validation, and testing datasets
train_datagen = ImageDataGenerator(rescale=1./255)  # Normalize pixel values between 0 and 1
val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

In [25]:
# Load training data
train_generator = train_datagen.flow_from_directory(
    os.path.join(DATASET_PATH, 'train'),
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary'
)

Found 5216 images belonging to 2 classes.


## 2. Visualize a Few Sample Images
### Visualize some of the training images to ensure everything looks correct.

In [ ]:
import matplotlib.pyplot as plt

# Display a few sample images from the training data
sample_images, sample_labels = next(train_generator)

plt.figure(figsize=(10, 10))
for i in range(9):
    plt.subplot(3, 3, i+1)
    plt.imshow(sample_images[i])
    plt.axis('off')
plt.show()

## 3. Build the GAN Model
### 3.1 Generator Model:
The generator will create synthetic images from random noise.

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers

def build_generator():
    model = tf.keras.Sequential()
    model.add(layers.Dense(128 * 16 * 16, activation="relu", input_dim=100))
    model.add(layers.Reshape((16, 16, 128)))
    model.add(layers.UpSampling2D())  # Output size: 32x32
    model.add(layers.Conv2D(128, kernel_size=3, padding="same"))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation("relu"))
    
    model.add(layers.UpSampling2D())  # Output size: 64x64
    model.add(layers.Conv2D(64, kernel_size=3, padding="same"))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation("relu"))
    
    model.add(layers.UpSampling2D())  # Output size: 128x128
    model.add(layers.Conv2D(1, kernel_size=3, padding="same"))
    model.add(layers.Activation("tanh"))  # Output in the range [-1, 1]

    return model


generator = build_generator()
generator.summary()


### 3.2 Discriminator Model:
The discriminator will classify real and synthetic images.

In [ ]:
def build_discriminator():
    model = tf.keras.Sequential()
    model.add(layers.Conv2D(64, kernel_size=3, strides=2, input_shape=(128, 128, 1), padding="same"))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Dropout(0.25))
    
    model.add(layers.Conv2D(128, kernel_size=3, strides=2, padding="same"))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Dropout(0.25))
    
    model.add(layers.Flatten())
    model.add(layers.Dense(1, activation='sigmoid'))  # Output 1 for real, 0 for fake
    
    return model

discriminator = build_discriminator()
discriminator.summary()

In [ ]:
train_generator = train_datagen.flow_from_directory(
    os.path.join(DATASET_PATH, 'train'),
    target_size=(128, 128),  # Make sure the images are resized to 128x128
    batch_size=BATCH_SIZE,
    class_mode='binary',
    color_mode='grayscale'  # Since the input expects single-channel (grayscale) images
)

val_generator = val_datagen.flow_from_directory(
    os.path.join(DATASET_PATH, 'val'),
    target_size=(128, 128),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    color_mode='grayscale'
)

test_generator = test_datagen.flow_from_directory(
    os.path.join(DATASET_PATH, 'test'),
    target_size=(128, 128),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    color_mode='grayscale'
)

## 4. Compile the Models
### Compile both the generator and discriminator models.

In [ ]:
# Compile discriminator
discriminator.compile(optimizer=tf.keras.optimizers.Adam(0.0002, 0.5), loss='binary_crossentropy', metrics=['accuracy'])

# GAN combines the generator and discriminator
def build_gan(generator, discriminator):
    discriminator.trainable = False  # Freeze the discriminator during generator training
    model = tf.keras.Sequential([generator, discriminator])
    model.compile(optimizer=tf.keras.optimizers.Adam(0.0002, 0.5), loss='binary_crossentropy')
    return model

gan = build_gan(generator, discriminator)
gan.summary()

## 5. Training the GAN

In [ ]:
import numpy as np

def train_gan(gan, generator, discriminator, dataset, epochs, batch_size):
    real_label = np.ones((batch_size, 1))
    fake_label = np.zeros((batch_size, 1))

    for epoch in range(epochs):
        for _ in range(batch_size):
            # Train the discriminator with real images
            real_images, _ = next(dataset)
            d_loss_real = discriminator.train_on_batch(real_images, real_label)

            # Train the discriminator with fake images
            noise = np.random.normal(0, 1, (batch_size, 100))
            fake_images = generator.predict(noise)
            d_loss_fake = discriminator.train_on_batch(fake_images, fake_label)

            # Train the generator via the GAN model
            g_loss = gan.train_on_batch(noise, real_label)

        print(f"Epoch {epoch+1}, D Loss Real: {d_loss_real[0]}, D Loss Fake: {d_loss_fake[0]}, G Loss: {g_loss}")

# Set training parameters
EPOCHS = 20
BATCH_SIZE = 32

# Start training
train_gan(gan, generator, discriminator, train_generator, EPOCHS, BATCH_SIZE)

## 6.Generate and Visualize Images
### Generate synthetic images from the generator and visualize them:

In [ ]:
# Generate synthetic images
noise = np.random.normal(0, 1, (10, 100))  # Generate random noise
generated_images = generator.predict(noise)

# Visualize the generated images
import matplotlib.pyplot as plt
for i in range(10):
    plt.imshow(generated_images[i].reshape(128, 128), cmap='gray')
    plt.axis('off')
    plt.show()

In [ ]:
import os

# Create results directory if it doesn't exist
os.makedirs('results', exist_ok=True)

def save_generated_images(epoch, generator, num_examples=10):
    noise = np.random.normal(0, 1, (num_examples, 100))
    generated_images = generator.predict(noise)
    
    # Rescale images from [-1, 1] to [0, 1]
    generated_images = (generated_images + 1) / 2.0

    plt.figure(figsize=(10, 1))
    for i in range(num_examples):
        plt.subplot(1, num_examples, i+1)
        plt.imshow(generated_images[i].reshape(128, 128), cmap='gray')
        plt.axis('off')
    plt.tight_layout()

    # Save generated images to results folder
    plt.savefig(f"results/generated_image_epoch_{epoch}.png")
    plt.close()


In [ ]:
# Create models directory if it doesn't exist
os.makedirs('models', exist_ok=True)

def save_model_weights(generator, discriminator, epoch):
    generator.save(f"models/generator_epoch_{epoch}.h5")
    discriminator.save(f"models/discriminator_epoch_{epoch}.h5")
